In [1]:
import magma as m
import mantle

This notebook demonstrates using native Python functions to construct circuits.

In [2]:
def clb(a, b, c, d):
    return (a & b) | (~c & d)

T = m.UInt[16]
class Combinational(m.Circuit):
    name = "Combinational"
    IO = ["a", m.In(T), "b", m.In(T), "c", m.Out(T)]
    @classmethod
    def definition(io):
        m.wire(clb(io.a, io.b, io.a, io.b), io.c)

In [3]:
from magma.simulator import PythonSimulator

simulator = PythonSimulator(Combinational)
a, b = 148, 41
simulator.set_value(Combinational.a, a)
simulator.set_value(Combinational.b, b)
simulator.evaluate()
assert simulator.get_value(Combinational.c) == clb(a, b, a, b)
print("Success!")

Success!


In [4]:
m.compile("build/Combinational", Combinational, output="coreir")
%cat build/Combinational.json

{"top":"global.Combinational",
"namespaces":{
  "global":{
    "modules":{
      "Combinational":{
        "type":["Record",[
          ["a",["Array",16,"BitIn"]],
          ["b",["Array",16,"BitIn"]],
          ["c",["Array",16,"Bit"]]
        ]],
        "instances":{
          "Invert16_inst0":{
            "genref":"coreir.not",
            "genargs":{"width":["Int",16]}
          },
          "and16_inst0":{
            "genref":"coreir.and",
            "genargs":{"width":["Int",16]}
          },
          "and16_inst1":{
            "genref":"coreir.and",
            "genargs":{"width":["Int",16]}
          },
          "or16_inst0":{
            "genref":"coreir.or",
            "genargs":{"width":["Int",16]}
          }
        },
        "connections":[
          ["self.a","Invert16_inst0.in"],
          ["and16_inst1.in0","Invert16_inst0.out"],
          ["self.a","and16_inst0.in0"],
          ["self.b","and16_inst0.in1"],
          ["or16_inst0.in0","and16_inst0.out"],
    